In [4]:
from database import *

In [2]:
ss = session
ss.rollback()
for img in ss.query(Image).all():
    if (not img.tag_refs) and img.tags:
        print(f'id={img.id}')
        tags = img.tags.split(' ')
        for tag in tags:
            tag_instance = Tag.get_unique(tag)
            img.tag_refs.append(tag_instance)
    else:
        print(f'not tags {img.id}')

ss.commit()


not tags 11
not tags 59
not tags 110
not tags 126
not tags 162
not tags 209
not tags 241
not tags 297
not tags 305
not tags 313
not tags 334
not tags 339
not tags 360
not tags 361
not tags 418
not tags 516
not tags 524
not tags 543
not tags 562
not tags 588
not tags 618
not tags 624
not tags 625
not tags 663
not tags 666
not tags 674
not tags 682
not tags 697
not tags 806
not tags 893
not tags 1042
not tags 1143
not tags 1148
not tags 1160
not tags 1196
not tags 1240
not tags 1381
not tags 1395
not tags 1426
not tags 1446
not tags 1453
not tags 1553
not tags 1623
not tags 1631
not tags 1866
not tags 1922
not tags 1928
not tags 1935
not tags 1936
not tags 1943
not tags 1949
not tags 1950
not tags 1971
not tags 1975
not tags 1999
not tags 2053
not tags 2069
not tags 2156
not tags 2159
not tags 2171
not tags 2187
not tags 2214
not tags 2237
not tags 2258
not tags 2263
not tags 2292
not tags 2337
not tags 2338
not tags 2341
not tags 2343
not tags 2384
not tags 2400
not tags 2454
not tags 2

In [6]:
from sqlalchemy import func
from sqlalchemy import select
func.count

In [4]:
ss.query(Tag)

In [13]:
r = ss.query(Tag.name, func.count('*')).select_from(Image).join(Images2Tags).join(Tag).group_by(Tag.name).all()

In [16]:
s = select(Tag, func.count('*')).join(Images2Tags).join(Image).group_by(Tag.name)

In [18]:
len(r)

36061

In [19]:
r = [i for i in r if i[1]>2]

In [21]:
len(r)

15821

In [28]:
r_sorted = sorted(r, key=lambda x: -x[1])

In [29]:
r_sorted[:10]

[('nipples', 63406),
 ('pussy', 48533),
 ('loli', 42099),
 ('no_bra', 41455),
 ('nopan', 39937),
 ('thighhighs', 38787),
 ('breasts', 33120),
 ('pussy_juice', 32295),
 ('pantsu', 28768),
 ('ass', 26607)]